# Separabilidad entre clases usando distancia Mahalanobis

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pandas as pd
import rasterio
from sklearn.model_selection import train_test_split
from rasterio.plot import show_hist, show
import fiona
from rasterio.tools.mask import mask
from pandas.util.testing import assert_frame_equal
from rasterio import Affine
import geopandas as gpd
pd.options.mode.use_inf_as_na = True
import scipy as sp
from scipy.spatial.distance import mahalanobis

In [2]:
df = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/dos.csv', index_col=0)
df = df.fillna(0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
class_1 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/1.csv', index_col=0).fillna(0)
class_2 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/2.csv', index_col=0).fillna(0)
class_3 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/3.csv', index_col=0).fillna(0)
class_4 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/4.csv', index_col=0).fillna(0)
class_5 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/5.csv', index_col=0).fillna(0)
class_6 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/6.csv', index_col=0).fillna(0)
class_7 = pd.read_csv('/Users/oscar/Code/Maestria/perc_remota/taller_3/datasets/7.csv', index_col=0).fillna(0)

class_1 = class_1[ (class_1['ndvi']!=0)]
class_2 = class_2[ (class_2['ndvi']!=0)]
class_3 = class_3[ (class_3['ndvi']!=0)]
class_4 = class_4[ (class_4['ndvi']!=0)]
class_5 = class_5[ (class_5['ndvi']!=0)]
class_6 = class_6[ (class_6['ndvi']!=0)]
class_7 = class_7[ (class_6['ndvi']!=0)]
no_data_class = df[(df['ndvi']==0)  & (df['elevation']==32767)]


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [261]:

separability =  pd.DataFrame(columns=['class','red', 'green', 'blue', 'nir','ndvi'])
separability['class'] = [1,2,3,4,5,6,7]
separability['red'] = [class_1['red'].mean(), class_2['red'].mean(),class_3['red'].mean(), class_4['red'].mean(),class_5['red'].mean(),class_6['red'].mean(),class_7['red'].mean()]
separability['green'] = [class_1['green'].mean(), class_2['green'].mean(),class_3['green'].mean(), class_4['green'].mean(),class_5['green'].mean(),class_6['green'].mean(),class_7['green'].mean()]
separability['blue'] = [class_1['blue'].mean(), class_2['blue'].mean(),class_3['blue'].mean(), class_4['blue'].mean(),class_5['blue'].mean(),class_6['blue'].mean(),class_7['blue'].mean()]
separability['nir'] = [class_1['nir'].mean(), class_2['nir'].mean(),class_3['nir'].mean(), class_4['nir'].mean(),class_5['nir'].mean(),class_6['nir'].mean(),class_7['nir'].mean()]
separability['ndvi'] = [class_1['ndvi'].mean(), class_2['ndvi'].mean(),class_3['ndvi'].mean(), class_4['ndvi'].mean(),class_5['ndvi'].mean(),class_6['ndvi'].mean(),class_7['ndvi'].mean()]
separability

,class,red,green,blue,nir,ndvi
0,1,27.287521,28.216323,28.843988,-43.775694,197.012343
1,2,23.513344,21.895254,21.962394,82.240832,95.996456
2,3,19.933449,20.322230,21.096989,152.237091,75.094098
3,4,26.862328,29.327179,31.492495,175.278566,72.367257
4,5,33.249652,36.682679,41.795792,154.806077,63.004053
5,6,62.103808,66.235559,86.230190,116.387108,31.096855
6,7,40.513417,44.581040,50.148409,177.418890,62.357154


In [250]:
def calculate_mahala_distance(class_id):
    pairsdict = {
    'classes1': np.full( shape=7, fill_value=class_id, dtype=np.int),
    'classes2': [1,2,3,4,5,6,7]
    }

    df = pd.DataFrame(separability)
    pairs = pd.DataFrame(pairsdict)

    pairs = pairs.merge(df, how='left', left_on=['classes1'], right_on=['class'])
    pairs = pairs.merge(df, how='left', left_on=['classes2'], right_on=['class'])
    pairs['vector1'] = pairs[['red_x','green_x','blue_x','nir_x','ndvi_x']].values.tolist()
    pairs['vector2'] = pairs[['red_y','green_y','blue_y','nir_y','ndvi_y']].values.tolist()
    mahala = pairs[['classes1', 'classes2', 'vector1', 'vector2']]
    covmx = df.drop(['class'], axis=1).cov()
    invcovmx = sp.linalg.inv(covmx)
    mahala['mahala_dist'] = mahala.apply(lambda x: (mahalanobis(x['vector1'], x['vector2'], invcovmx)), axis=1)
    return mahala

In [251]:
calculate_mahala_distance(1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,1,1,"[27.28752062065483, 28.216323462875458, 28.843...","[27.28752062065483, 28.216323462875458, 28.843...",0.000000
1,1,2,"[27.28752062065483, 28.216323462875458, 28.843...","[23.513343670932183, 21.89525429907569, 21.962...",3.417843
2,1,3,"[27.28752062065483, 28.216323462875458, 28.843...","[19.933449282781208, 20.322230166882132, 21.09...",3.200420
3,1,4,"[27.28752062065483, 28.216323462875458, 28.843...","[26.862328320303888, 29.32717896256726, 31.492...",2.948095
4,1,5,"[27.28752062065483, 28.216323462875458, 28.843...","[33.2496522353248, 36.68267881778073, 41.79579...",3.458991
5,1,6,"[27.28752062065483, 28.216323462875458, 28.843...","[62.103808329943135, 66.23555922542728, 86.230...",3.459434
6,1,7,"[27.28752062065483, 28.216323462875458, 28.843...","[40.51341723373146, 44.58104010138931, 50.1484...",3.375275


In [253]:
calculate_mahala_distance(2)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,2,1,"[23.513343670932183, 21.89525429907569, 21.962...","[27.28752062065483, 28.216323462875458, 28.843...",3.417843
1,2,2,"[23.513343670932183, 21.89525429907569, 21.962...","[23.513343670932183, 21.89525429907569, 21.962...",0.000000
2,2,3,"[23.513343670932183, 21.89525429907569, 21.962...","[19.933449282781208, 20.322230166882132, 21.09...",2.903174
3,2,4,"[23.513343670932183, 21.89525429907569, 21.962...","[26.862328320303888, 29.32717896256726, 31.492...",3.228864
4,2,5,"[23.513343670932183, 21.89525429907569, 21.962...","[33.2496522353248, 36.68267881778073, 41.79579...",3.443622
5,2,6,"[23.513343670932183, 21.89525429907569, 21.962...","[62.103808329943135, 66.23555922542728, 86.230...",3.442702
6,2,7,"[23.513343670932183, 21.89525429907569, 21.962...","[40.51341723373146, 44.58104010138931, 50.1484...",3.192894


In [255]:
calculate_mahala_distance(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,3,1,"[19.933449282781208, 20.322230166882132, 21.09...","[27.28752062065483, 28.216323462875458, 28.843...",3.200420
1,3,2,"[19.933449282781208, 20.322230166882132, 21.09...","[23.513343670932183, 21.89525429907569, 21.962...",2.903174
2,3,3,"[19.933449282781208, 20.322230166882132, 21.09...","[19.933449282781208, 20.322230166882132, 21.09...",0.000000
3,3,4,"[19.933449282781208, 20.322230166882132, 21.09...","[26.862328320303888, 29.32717896256726, 31.492...",1.453034
4,3,5,"[19.933449282781208, 20.322230166882132, 21.09...","[33.2496522353248, 36.68267881778073, 41.79579...",3.115861
5,3,6,"[19.933449282781208, 20.322230166882132, 21.09...","[62.103808329943135, 66.23555922542728, 86.230...",3.119894
6,3,7,"[19.933449282781208, 20.322230166882132, 21.09...","[40.51341723373146, 44.58104010138931, 50.1484...",3.420770


In [256]:
calculate_mahala_distance(4)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,4,1,"[26.862328320303888, 29.32717896256726, 31.492...","[27.28752062065483, 28.216323462875458, 28.843...",2.948095
1,4,2,"[26.862328320303888, 29.32717896256726, 31.492...","[23.513343670932183, 21.89525429907569, 21.962...",3.228864
2,4,3,"[26.862328320303888, 29.32717896256726, 31.492...","[19.933449282781208, 20.322230166882132, 21.09...",1.453034
3,4,4,"[26.862328320303888, 29.32717896256726, 31.492...","[26.862328320303888, 29.32717896256726, 31.492...",0.000000
4,4,5,"[26.862328320303888, 29.32717896256726, 31.492...","[33.2496522353248, 36.68267881778073, 41.79579...",3.056175
5,4,6,"[26.862328320303888, 29.32717896256726, 31.492...","[62.103808329943135, 66.23555922542728, 86.230...",3.051715
6,4,7,"[26.862328320303888, 29.32717896256726, 31.492...","[40.51341723373146, 44.58104010138931, 50.1484...",2.290888


In [257]:
calculate_mahala_distance(5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,5,1,"[33.2496522353248, 36.68267881778073, 41.79579...","[27.28752062065483, 28.216323462875458, 28.843...",3.458991
1,5,2,"[33.2496522353248, 36.68267881778073, 41.79579...","[23.513343670932183, 21.89525429907569, 21.962...",3.443622
2,5,3,"[33.2496522353248, 36.68267881778073, 41.79579...","[19.933449282781208, 20.322230166882132, 21.09...",3.115861
3,5,4,"[33.2496522353248, 36.68267881778073, 41.79579...","[26.862328320303888, 29.32717896256726, 31.492...",3.056175
4,5,5,"[33.2496522353248, 36.68267881778073, 41.79579...","[33.2496522353248, 36.68267881778073, 41.79579...",0.000000
5,5,6,"[33.2496522353248, 36.68267881778073, 41.79579...","[62.103808329943135, 66.23555922542728, 86.230...",3.464092
6,5,7,"[33.2496522353248, 36.68267881778073, 41.79579...","[40.51341723373146, 44.58104010138931, 50.1484...",3.326239


In [258]:
calculate_mahala_distance(6)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,6,1,"[62.103808329943135, 66.23555922542728, 86.230...","[27.28752062065483, 28.216323462875458, 28.843...",3.459434
1,6,2,"[62.103808329943135, 66.23555922542728, 86.230...","[23.513343670932183, 21.89525429907569, 21.962...",3.442702
2,6,3,"[62.103808329943135, 66.23555922542728, 86.230...","[19.933449282781208, 20.322230166882132, 21.09...",3.119894
3,6,4,"[62.103808329943135, 66.23555922542728, 86.230...","[26.862328320303888, 29.32717896256726, 31.492...",3.051715
4,6,5,"[62.103808329943135, 66.23555922542728, 86.230...","[33.2496522353248, 36.68267881778073, 41.79579...",3.464092
5,6,6,"[62.103808329943135, 66.23555922542728, 86.230...","[62.103808329943135, 66.23555922542728, 86.230...",0.000000
6,6,7,"[62.103808329943135, 66.23555922542728, 86.230...","[40.51341723373146, 44.58104010138931, 50.1484...",3.328650


In [259]:
calculate_mahala_distance(7)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,classes1,classes2,vector1,vector2,mahala_dist
0,7,1,"[40.51341723373146, 44.58104010138931, 50.1484...","[27.28752062065483, 28.216323462875458, 28.843...",3.375275
1,7,2,"[40.51341723373146, 44.58104010138931, 50.1484...","[23.513343670932183, 21.89525429907569, 21.962...",3.192894
2,7,3,"[40.51341723373146, 44.58104010138931, 50.1484...","[19.933449282781208, 20.322230166882132, 21.09...",3.420770
3,7,4,"[40.51341723373146, 44.58104010138931, 50.1484...","[26.862328320303888, 29.32717896256726, 31.492...",2.290888
4,7,5,"[40.51341723373146, 44.58104010138931, 50.1484...","[33.2496522353248, 36.68267881778073, 41.79579...",3.326239
5,7,6,"[40.51341723373146, 44.58104010138931, 50.1484...","[62.103808329943135, 66.23555922542728, 86.230...",3.328650
6,7,7,"[40.51341723373146, 44.58104010138931, 50.1484...","[40.51341723373146, 44.58104010138931, 50.1484...",0.000000
